In [12]:
# Import libraries
import pandas as pd
import numpy as np

In [13]:
# Import 2 tables from IMDb datasets that we need
title_basics = pd.read_csv('titlebasics.csv')
title_ratings = pd.read_csv('title.ratings.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
# Select only movies from the title_basics table
movies = title_basics[(title_basics.titleType == 'movie')].copy()

In [15]:
# Set the indicies for our 2 tables to the IMDb key for movies
movies.set_index('tconst', inplace=True)
title_ratings.set_index('tconst', inplace=True)
# Joint the 2 tables by tconst, the IMDb key for all movies
movies_with_rating = movies.join(title_ratings, how='inner')

In [16]:
# Take only the top 10,000 movies, where we rank movies by the number of votes they have received
top_10000_movies = movies_with_rating.sort_values(by='numVotes', ascending=False)[:10000]
movies_index = top_10000_movies.index

In [17]:
#from tqdm import tqdm
#from time import sleep
#from imdb import IMDb

#ia = IMDb()

In [18]:
#keywords_dict = {}
#for movie_index in tqdm(movies_index):
#    sleep(1)
#    try:
#        keywords_dict[movie_index] = ia.get_movie_keywords(movie_index[2:])['data']['keywords']
#    except:
#        keywords_dict[movie_index] = ''

In [19]:
#keywords = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in keywords_dict.items() ])).transpose()
#keywords = keywords.apply(lambda x: ','.join(x.dropna()), axis=1)
#keywords = pd.DataFrame(keywords)
#keywords.rename(columns={0:'keywords'}, inplace=True)

In [20]:
# Save the plots to a CSV
#keywords.to_csv(path_or_buf='keywords.csv')

In [21]:
# Load our keywords for the CSV
keywords = pd.read_csv('keywords.csv')
keywords.rename(columns={'Unnamed: 0':'tconst'}, inplace=True)
keywords.set_index('tconst', inplace=True)

In [22]:
# Join to our table
movies = top_10000_movies.join(keywords, how='inner')

In [23]:
movies = movies[movies.keywords.notnull()]

In [24]:
movies.reset_index(inplace=True)
movies = movies[['primaryTitle', 'keywords']]

In [25]:
# Processing Keywords
keywords = movies['keywords'].tolist()

from nltk.tokenize import word_tokenize
keywords = [word_tokenize(keyword.lower()) for keyword in keywords]

In [26]:
def no_commas(doc):
    no_commas = [t for t in doc if t!=',']
    return(no_commas)

keywords = [no_commas(kw) for kw in keywords]
processed_keywords = keywords

In [27]:
from gensim.corpora.dictionary import Dictionary
dictionary = Dictionary(processed_keywords) # create a dictionary of words from our keywords

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [28]:
corpus = [dictionary.doc2bow(doc) for doc in processed_keywords] 
#create corpus where the corpus is a bag of words for each document

In [29]:
from gensim.models.tfidfmodel import TfidfModel
tfidf = TfidfModel(corpus) #create tfidf model of the corpus

In [30]:
from gensim.similarities import MatrixSimilarity
# Create the similarity data structure. This is the most important part where we get the similarities between the movies.
sims = MatrixSimilarity(tfidf[corpus], num_features=len(dictionary))

In [31]:
def movie_recommendation(movie_title, number_of_hits=10):
    movie = movies.loc[movies.primaryTitle==movie_title] # get the movie row
    keywords = movie['keywords'].iloc[0].split(',') #get the keywords as a Series (movie['keywords']),
    # get just the keywords string ([0]), and then convert to a list of keywords (.split(',') )
    query_doc = keywords #set the query_doc to the list of keywords
    
    query_doc_bow = dictionary.doc2bow(query_doc) # get a bag of words from the query_doc
    query_doc_tfidf = tfidf[query_doc_bow] #convert the regular bag of words model to a tf-idf model where we have tuples
    # of the movie ID and it's tf-idf value for the movie

    similarity_array = sims[query_doc_tfidf] # get the array of similarity values between our movie and every other movie. 
    #So the length is the number of movies we have. To do this, we pass our list of tf-idf tuples to sims.

    similarity_series = pd.Series(similarity_array.tolist(), index=movies.primaryTitle.values) #Convert to a Series
    top_hits = similarity_series.sort_values(ascending=False)[1:number_of_hits+1] 
    #get the top matching results, i.e. most similar movies; start from index 1 because every movie is most similar to itself

    #print the words with the highest tf-idf values for the provided movie:
    sorted_tfidf_weights = sorted(tfidf[corpus[movie.index.values.tolist()[0]]], key=lambda w: w[1], reverse=True)
    print('The top 5 words associated with this movie by tf-idf are: ')
    for term_id, weight in sorted_tfidf_weights[:5]:
        print(" '%s' with a tf-idf score of %.3f" %(dictionary.get(term_id), weight))
    
    # Print the top matching movies
    print("Our top %s most similar movies for movie %s are:" %(number_of_hits, movie_title))
    for idx, (movie,score) in enumerate(zip(top_hits.index, top_hits)):
        print("%d %s with a similarity score of %.3f" %(idx+1, movie, score))

In [33]:
movie_recommendation('Titanic')

The top 5 words associated with this movie by tf-idf are: 
 'abusive-fiance' with a tf-idf score of 0.072
 'below-decks' with a tf-idf score of 0.072
 'berth' with a tf-idf score of 0.072
 'blueprints' with a tf-idf score of 0.072
 'bolts' with a tf-idf score of 0.072
Our top 10 most similar movies for movie Titanic are:
1 A Night to Remember with a similarity score of 0.234
2 Poseidon with a similarity score of 0.122
3 Titanic with a similarity score of 0.119
4 Beyond the Poseidon Adventure with a similarity score of 0.100
5 The Poseidon Adventure with a similarity score of 0.077
6 The Hindenburg with a similarity score of 0.076
7 Speed 2: Cruise Control with a similarity score of 0.073
8 Airport '77 with a similarity score of 0.071
9 Juggernaut with a similarity score of 0.070
10 Godzilla with a similarity score of 0.070
